In [20]:
import pandas as pd
import os
import numpy as np

In [21]:
# TODO: Write the model name
model = 'Llama-3.1-8B'

In [22]:
# load the data into a single dataframe
df = pd.DataFrame()
for file in os.listdir(f'decision_datasets/{model}/'):
    if file.endswith('.csv'):
        df = pd.concat([df, pd.read_csv(f'decision_datasets/{model}/' + file)], ignore_index=True)

In [23]:
# have a look at the data
df

,id,model,temperature,seed,timestamp,control_options,control_option_order,control_answer,control_extraction,control_decision,treatment_options,treatment_option_order,treatment_answer,treatment_extraction,treatment_decision,status,error_message,individual_score,weight,bias
0,25890,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,42,2024-10-05 16:47:06,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[0, 1, 2, 3, 4, 5, 6]",I would select:\n\nOption 2: Good\n\nAn on-tim...,Option 2,1.0,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[0, 1, 2, 3, 4, 5, 6]",I would select:\n\nOption 6: Bad\n\nA late del...,Option 6,5.0,OK,NaN,0.800000,1.0,FramingEffect
1,25891,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,47,2024-10-05 16:47:09,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 5: Somewhat good\n\n...,Option 5,2.0,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 3: Somewhat bad\n\nA...,Option 3,4.0,OK,NaN,0.500000,1.0,FramingEffect
2,25892,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,52,2024-10-05 16:47:11,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[0, 1, 2, 3, 4, 5, 6]",I would select:\n\nOption 6: Bad\n\nAn on-time...,Option 6,5.0,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[0, 1, 2, 3, 4, 5, 6]",I would select:\n\nOption 6: Bad\n\nA late del...,Option 6,5.0,OK,NaN,0.000000,1.0,FramingEffect
3,25893,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,57,2024-10-05 16:47:14,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 5: Somewhat good\n\n...,Option 5,2.0,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 2: Bad\n\nA late del...,Option 2,5.0,OK,NaN,0.600000,1.0,FramingEffect
4,25894,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,62,2024-10-05 16:47:17,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[0, 1, 2, 3, 4, 5, 6]",I would select:\n\nOption 2: Good\n\nAn on-tim...,Option 2,1.0,"['Very good', 'Good', 'Somewhat good', 'Neithe...","[0, 1, 2, 3, 4, 5, 6]",I would select:\n\nOption 6: Bad\n\nA 15% late...,Option 6,5.0,OK,NaN,0.800000,1.0,FramingEffect
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,8985,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,67,2024-10-05 16:38:52,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 6: Likely negative a...,Option 6,1.0,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 7: Very likely negat...,Option 7,0.0,OK,NaN,0.833333,1.0,NegativityBias
29996,8986,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,72,2024-10-05 16:38:56,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 7: Very likely negat...,Option 7,0.0,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 7: Very likely negat...,Option 7,0.0,OK,NaN,1.000000,1.0,NegativityBias
29997,8987,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,77,2024-10-05 16:38:59,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]",I would select:\n\nOption 2: Likely negative a...,Option 2,1.0,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]",I would select:\n\nOption 6: Likely positive a...,Option 6,5.0,OK,NaN,0.000000,1.0,NegativityBias
29998,8988,meta-llama/Meta-Llama-3.1-8B-Instruct,0.0,82,2024-10-05 16:39:02,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]",I would choose:\n\nOption 7: Very likely negat...,Option 7,0.0,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]",I would select:\n\nOption 6: Likely negative a...,Option 6,1.0,OK,NaN,1.000000,1.0,NegativityBias


In [24]:
# check for missing decisions: should be 1000 for each bias
df.groupby('bias')['id'].count()

bias
Anchoring                      1000
Anthropomorphism               1000
AvailabilityHeuristic          1000
BandwagonEffect                1000
ConfirmationBias               1000
Conservatism                   1000
DispositionEffect              1000
EndowmentEffect                1000
EscalationOfCommitment         1000
FramingEffect                  1000
FundamentalAttributionError    1000
HaloEffect                     1000
HindsightBias                  1000
HyperbolicDiscounting          1000
IllusionOfControl              1000
InGroupBias                    1000
InformationBias                1000
LossAversion                   1000
MentalAccounting               1000
NegativityBias                 1000
NotInventedHere                1000
OptimismBias                   1000
PlanningFallacy                1000
Reactance                      1000
RiskCompensation               1000
SelfServingBias                1000
SocialDesirabilityBias         1000
StatusQuoBias          

In [25]:
# get the number of completed tests
# df[df['status'] == 'OK'].groupby('bias')['id'].count().sort_values(ascending=False)
# in percentage
(df[df['status'] == 'OK'].groupby('bias')['id'].count().sort_values(ascending=False) / df.groupby('bias')['id'].count() * 100).sort_values(ascending=False)

bias
SurvivorshipBias               100.0
BandwagonEffect                100.0
StatusQuoBias                   99.9
FramingEffect                   99.9
EndowmentEffect                 99.6
InformationBias                 99.6
DispositionEffect               97.6
PlanningFallacy                 97.6
NotInventedHere                 97.1
NegativityBias                  96.2
SelfServingBias                 95.1
ConfirmationBias                95.0
Anthropomorphism                93.4
HyperbolicDiscounting           92.9
MentalAccounting                92.8
InGroupBias                     90.4
HaloEffect                      89.3
LossAversion                    84.7
Conservatism                    83.6
OptimismBias                    79.5
IllusionOfControl               68.3
EscalationOfCommitment          58.9
Anchoring                       58.0
AvailabilityHeuristic           57.3
FundamentalAttributionError     41.3
RiskCompensation                27.4
Stereotyping                    1

In [26]:
# evaluate the biasedness of the model
df_success = df[df['status'] == 'OK']

df_success.groupby(df_success['bias']).apply(lambda x: np.average(x['individual_score'], weights=x['weight'])).sort_values(ascending=False)

/tmp/ipykernel_543926/3520039201.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_success.groupby(df_success['bias']).apply(lambda x: np.average(x['individual_score'], weights=x['weight'])).sort_values(ascending=False)


bias
InGroupBias                    0.826143
Anchoring                      0.543836
InformationBias                0.542052
FramingEffect                  0.535802
PlanningFallacy                0.510538
NegativityBias                 0.443798
HaloEffect                     0.411310
SurvivorshipBias               0.392817
Stereotyping                   0.365982
LossAversion                   0.261971
AvailabilityHeuristic          0.234492
Conservatism                   0.211882
HindsightBias                  0.180849
IllusionOfControl              0.169043
HyperbolicDiscounting          0.159706
SelfServingBias                0.141220
BandwagonEffect                0.095800
NotInventedHere                0.061878
OptimismBias                   0.043622
EscalationOfCommitment         0.014677
RiskCompensation               0.013472
EndowmentEffect                0.007017
ConfirmationBias               0.006160
FundamentalAttributionError    0.004964
MentalAccounting              -0.01

In [29]:
df.to_csv(f'decision_datasets/{model}.csv', index=False)